![image](images/roboticsdevday.jpeg)

# Robot Arm Motion Planning with PyRoboPlan

Welcome! In this session you will learn some of the basics of motion planning with robot manipulators (often referred to as robot arm).

We will be using the PyRoboPlan library, which is a Python based educational tool.

https://github.com/sea-bass/pyroboplan

## Who Is Sebastian Castro?

I'm a robotics engineer and applied scientist at [The AI Institute](https://theaiinstitute.com/).

A brief summary of my career:
* BS + MS Mech. Eng. at [Cornell University](https://www.cornell.edu/) (2006-2011)
* Technical support, training, and robotics education marketing at [MathWorks](https://www.mathworks.com/) (2011-2019)
* Robotics software engineer at
  * [MIT CSAIL](https://www.csail.mit.edu/) (2020-2022)
  * [Boston Dynamics](https://bostondynamics.com/) (2022-2023)
  * [PickNik Robotics](https://picknik.ai/) (2023-2024)
  
I also make educational content which I share on [GitHub](https://github.com/sea-bass) and my personal Web site, https://roboticseabass.com/.

![image](images/scastro_career.png)

## Setup
Open up a Web shell and wait for the automatic installation to complete (this can take a few minutes). If this did not run automatically for some reason, you can run the `~/course_install.sh` script yourself.

![image](images/construct_web_shell.png)

To run and visualize examples, take the following steps:

1. Open a new Web shell in your rosject.
2. `cd ~/pyroboplan/examples`
3. `python3 rrt_panda.py`

As instructed by the script, open a Web browser (Google Chrome) so we can use the MeshCat visualizer for the examples.

1. Open a new Web shell terminal in your rosject.
2. `sudo google-chrome --no-sandbox`
3. Open the "Graphical tools" section in your rosject.
4. In Chrome, go to the MeshCat URL displayed on your console. This will likely be `http://127.0.0.1:7000/static/`, but could be different as you run more examples.

![image](images/construct_browser.png)

All the examples should be available from your Code Editor, under the `pyroboplan/examples` folder. As seen below, you can follow along and read the code!

![image](images/construct_ide.png)

###  Robot Modeling Basics

We will start by discussing the basics of robot modeling and kinematics.

PyRoboPlan turn uses the [Pinocchio](https://github.com/stack-of-tasks/pinocchio) library under the hood to do all the heavy lifting.

Let's start by modeling a robot manually in Pinocchio. A robot is composed or *rigid bodies* and *joints* that define motion between them.

```
python3 intro_pinocchio/intro_pinocchio_manual.py
```

Notice that you can use the navigation tree on the right of the MeshCat visualizer to enable and disable components to visualize. This will be useful in the following examples.

![image](images/intro_pinocchio_manual.png)

You can also import robot models from [URDF files](https://wiki.ros.org/urdf), which are commonly used in many ROS tools, such as Gazebo and MoveIt!

Here is an example that loads a Universal Robots UR5 manipulator.

```
python3 intro_pinocchio/intro_pinocchio_ur5_sim.py
```

The imported model should look as follows in MeshCat. You may need to disable collision geometry visualization using the navigation tree on the right.

![image](images/intro_pinocchio_ur5.png)

Pinocchio provides several tools necessary to perform motion planning, such as forward kinematics, forward and inverse dynamics, and Jacobians.

Collision checking is also very important for motion planning. Pinocchio uses the [HPP-FCL](https://github.com/humanoid-path-planner/hpp-fcl) library to handle this.

This time around, we will use the Franka Emika Panda manipulator to move along a hard-coded path and check for collisions.

```
python3 collision_along_path.py
```

Note you may need to run the example multiple times (and refresh your browser), since some paths will be collision-free.

![image](images/collision_along_path.png)

## Motion Planning Components

Now, we will dig into a few detailed examples of common motion planning components related to robot manipulators.

The key idea behind motion planning is "move from A to B without bad things happening".

Classically, the components that make up motion planning are as shown below. We will see examples of some of these throughout the rest of the session.

![image](images/motion_planning_components.png)

### Inverse Kinematics

Inverse kinematics (IK) is the problem of finding joint positions that put some coordinate frame on the robot at a specific pose in the real world.

For example, if you want to position the gripper over an object, you will need to solve IK to figure out where the joints should move.

There are many approaches to solving IK. You can solve it *analytically* using the geometry of the robot, but this often does not work because the model has too many degrees of freedom, or there are other constraints like joint limits and collisions that need to be accounted for. So there are many *numerical methods* to do this.

One common numerical method is the Inverse Jacobian method, described in detail [here](https://homes.cs.washington.edu/~todorov/courses/cseP590/06_JacobianMethods.pdf).

```
python3 differential_ik.py
```

![image](images/manip_jacobian_ik.png)

![image](images/differential_ik.png)

### Sampling-based motion planning

There are two common motion planning approaches that use sampling to generate a graph and then search along that graph for collision-free paths.

![image](images/rrt_2dof_config_space.png)

* **Probabilistic Road Maps (PRM)**: This is a *multi-query planner*, meaning that once you generate a graph of randomly-sampled points and connect them together, you can search multiple times using that same graph. It's a lot of up-front construction time for fast and reusable motion planning, which can be useful in static environments.

* **Rapidly-exploring Random Trees (RRT)**: This is a *single-query planner*, meaning that the tree (which is a type of graph) is "grown" from the start to the goal and then discarded. RRTs, as their name suggest, are faster and more sample-efficient than PRMs, but they are single-use. Sometimes, RRTs are *bidirectional*, meaning two parallel trees are grown from both start and goal and they are connected somewhere in the middle.

Let's try some examples with a simple 2-DOF robot.

```
python3 prm_2dof.py
python3 rrt_2dof.py
```

After that, let's try it with a higher degree of freedom robot!

```
python3 prm_panda.py
python3 rrt_panda.py
```

![image](images/manip_sampling_planning.png)

You may have noticed that the paths don't look great... so how can we change the settings in our planners to get better results?

* Try RRT* or PRM* variants, which sacrifice some performance to "rewire" the graphs and get better paths.
* Apply path shortcutting, which samples along an already-computed path and looks for shortcuts.

![image](images/manip_rrt_comparison.png)

### Trajectory generation

The path planners above only return *paths*; that is, a list of waypoints, but no information about timing.

Real robots have limits such as velocity, acceleration, and torque limits. So paths need to be converted to *trajectories*, which describe how those paths are followed over time.

There are many ways of representing trajectories. Two popular ones are:

* **Trapezoidal velocity trajectories:** Used in industrial settings because they are easy to implement and have guarantees on joint velocity/acceleration limits.
* **Polynomial trajectories:** Usually cubic (3rd order) or quintic (5th order). These trajectories provide smooth profiles that can pass through waypoints at specific velocities and accelerations, but they may "overshoot" their waypoints and therefore need to be checked carefully against joint limits.

In the following example, you can change the settings and look at results from both kinds of trajectory generation methods.

```
python3 trajectory_generation.py
```

![image](images/trajectory_generation.png)

### Trajectory Optimization

You can also use nonlinear optimization to generate trajectories. There are two main types of methods:

* **Direct transcription:** Here, you usually discretize your robot dynamics into small steps and optimize for each individual trajectory point. It's powerful, but can get very slow with long segments.
* **Direct collocation:** What if we take the representations in the previous section, like polynomial trajectories, and instead optimize over the boundary conditions of the trajectories? These are more widely used in practice, even though they do limit the representation space of trajectories.

The great thing about nonlinear optimization is that you can introduce arbitrary constraints into the problem. This includes things like position, velocity, acceleration, or even jerk limits. It can also be dynamic constraints like force/torque, limits, or even geometric constraints like collision avoidance!

Given the flexibility of nonlinear optimization, you can actually directly optimize a simple straight-line path and  get a valid trajectory out of it.

```
python3 trajectory_optimization.py
```

![image](images/trajectory_optimization.png)

### Trajectory Optimization to Improve Planning

One thing about trajectory optimization is that you should be careful of local minima; the initial guess for optimization is very important to getting good results!

One thing you can do is start with a path from a planner, such as an RRT, and then optimize it to smooth it out. This can be a more principled alternative to sampling-based path shortcutting.

```
python3 optimize_rrt_path.py
```

![image](images/trajectory_optimization_rrt.png)

### Cartesian planning

You may have noticed that a straight-line path in joint space is not necessarily a straight line in the real world. Sometimes, you need robots to follow paths in task space, for example drawing, welding, or wiping a table.

To do this, you can actually perform *Cartesian planning*, in which you convert straight-line paths (or trajectories) to joint-space paths using inverse kinematics.

```
python3 cartesian_planning.py
```

![image](images/manip_cartesian_path.png)


## Conclusion

Thank you for attending this skill learning session!

You have now seen some of the basics of motion planning for robot manipulators using [PyRoboPlan](https://github.com/sea-bass/pyroboplan).

However, this tool is educational, so it would be challenging to get high-performing motion plans from Python, as well as connecting the outputs of these plans to run on a real robot.

Many other tools in the robotics and ROS ecosystem exist for motion planning. Some that you should know include:

* [**MoveIt:**](https://github.com/moveit/moveit2) Contains good support for ROS enabled robots, including a direct connection to `ros2_control`.
* [**Tesseract:**](https://github.com/tesseract-robotics/tesseract) Inspired by MoveIt, but redesigned to be better suited for trajectory optimization and prototyping using easy-to-use Python bindings.
* [**Drake:**](https://github.com/RobotLocomotion/drake) Contains state-of-the-art optimization based planning capabilities, but the ROS interface is experimental.
* [**Pinocchio:**](https://github.com/stack-of-tasks/pinocchio) Includes all the tools you need to model a robot, but does not inherently have any motion planning capabilities. However, many tools are built on Pinocchio.

Enjoy the journey!